In [1]:
%pip install --upgrade "crewai[tools]"

In [ ]:
DB_URL = "52.118.208.226"
DB_PORT = "8020"
DB_NAME = "mydb"
DB_USER = "myuser"
DB_PASSWORD = ""

WX_URL= "https://us-south.ml.cloud.ibm.com"
WX_API_KEY = ""

In [ ]:
from crewai_tools import NL2SQLTool
from crewai import LLM
import os, json
from crewai import Agent, Crew, Task, Process

In [3]:
nl2sql = NL2SQLTool(db_uri=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}:{DB_PORT}/{DB_NAME}")

In [4]:
# Initialize LLM
llm_db = LLM(
    api_key = WX_API_KEY,
    api_base = WX_URL,
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 15000,
        "temperature": 0,
        "repetition_penalty": 1.05
    }
)

# Initialize LLM
llm_ba = LLM(
    api_key = WX_API_KEY,
    api_base = WX_URL,
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "sample",
        "max_new_tokens": 15000,
        "temperature": 0.8,
        "repetition_penalty": 0.1
    }
)

def create_dbagent() -> Agent:
    return Agent(
        role="Database Analyst",
        goal=(
            "Eksekusi SQL dan tampilkan hasilnya. "
            "Saat memanggil NL2SQLTool, PASTIKAN Action Input hanya "
            "berupa {\"sql_query\": <SQL>}."
        ),
        backstory=(
            "Seorang analis data veteran yang handal membuat laporan ad-hoc."
        ),
        llm=llm_db,
        allow_delegation=False,
        tools=[nl2sql],
    )

def create_researcher() -> Agent:
    return Agent(
        role="Business Analyst",
        goal=(
            """Analisis data dan pertanyaan user untuk membuat report.
            pahami pertanyaan user dengan snipet dari tabel sebagai berikut:
            transaction_timestamp,customer_id,customer_name,customer_birthday,customer_marriage_status,customer_savings_value,transaction_value,transaction_type,spending_location_or_category,estimated_annual_income,total_spending_last_month,most_common_spending_category,has_existing_loan,loan_product_interest_level,customer_segment
            2024-03-01 08:15:00,CUST001,Alice Wonderland,1990-05-20,Single,15230500.0,-55200.0,Debit,Grocery Store FreshFoods,75000000,1200000,Groceries,False,Medium,Young Professional
            2024-08-15 11:00:00,CUST069,Iron Man,1970-05-29,Single,400000000.0,-1500000.0,Debit,Components Purchase,400000000,12000000,Other,False,nan,High Net Worth
            Delegasikan eksekusi ke Database Analyst."""
        ),
        backstory="Kamu adalah seorang expert di bidang data analisis dan membuat report dari database.",
        llm=llm_ba,
        allow_delegation=True,
    )

def create_generation_task(agent, natural_query):
    return Task(
        description = f"""{natural_query}.""",
        expected_output = """Hasil dari database setelah running query""",
        agent = agent
    )

def create_analysis_task(agent, natural_query):
    return Task(
        description = f"""{natural_query}.""",
        expected_output = """Table atau hasil analisis dari report secara text""",
        agent=agent
    )

### Running both Analysis task and Generation task

In [5]:
# user_question = "Berapa banyak segmentasi customer yang masuk ke dalam kategori Young Professional?"
# user_question = "Berapa persentase pelanggan yang statusnya Single, Married, dan Divorced?"
user_question = "Siapa saja pelanggan yang total pengeluaran bulan lalu di atas 3 juta rupiah"

research_agent=create_researcher()
query_agent=create_dbagent()

generation_task = create_generation_task(research_agent, user_question)
analysis_task = create_analysis_task(research_agent, user_question)

generation_crew = Crew(
    agents=[research_agent, query_agent],
    tasks=[analysis_task, generation_task],
    verbose=True
    )

In [6]:
crew_output = generation_crew.kickoff()

# Accessing the crew output
print(f"Raw Output: {crew_output.raw}")

if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")

if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")

print(f"Tasks Output: {crew_output.tasks_output}")
print(f"/n/nToken Usage: {crew_output.token_usage}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9ead9c2a-d6fd-48cb-8ffb-8ae183c7a669                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0c8c8a01-262b-4008-a856-472f96c05339
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0c8c8a01-262b-4008-a856-472f96c05339
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress

# Agent: Business Analyst
## Task: Siapa saja pelanggan yang total pengeluaran bulan lalu di atas 3 juta rupiah.


🤖 Agent: Business Analyst
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 0c8c8a01-262b-4008-a856-472f96c05339
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Database Analyst
│   │       Status: In Progress
│   │   └── 🔧 Used Ask question to coworker (1)
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
└── 📋 Task: 9879771b-3b9e-4a86-9773-33cf73391311
       Status: Executing Task...
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    ├── 🤖 Agent: Database Analyst
    │       Status: In Progress
    │   └── 🔧 Used Delegate work to coworker (3)
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    ├── 🤖 Agent: Database Analyst
    │       Status: In Progress
    │   └── 🔧 Used Ask question to coworker (3)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 0c8c8a01-262b-4008-a856-472f96c05339
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Database Analyst
│   │       Status: In Progress
│   │   └── 🔧 Used Ask question to coworker (1)
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
└── 📋 Task: 9879771b-3b9e-4a86-9773-33cf73391311
       Assigned to: Business Analyst
       Status: ✅ Completed
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    ├── 🤖 Agent: Database Analyst
    │       Status: In Progress
    │   └── 🔧 Used Delegate work to coworker (3)
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    ├── 🤖 Agent: Database Analyst
    │       Status: In Progress
    │   └── 🔧 Used Ask question to coworker (3)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9879771b-3b9e-4a86-9773-33cf73391311                                                                     │
│  Agent: Business Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9ead9c2a-d6fd-48cb-8ffb-8ae183c7a669                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Raw Output: Berikut adalah daftar pelanggan yang total pengeluaran bulan lalu di atas 3 juta rupiah: 
- Iron Man (CUST069) dengan total pengeluaran 12.000.000
- Tony Stark (CUST056) dengan total pengeluaran 15.000.000.000 
- Queen Amidala (CUST017) dengan total pengeluaran 4.000.000.000 
- Cruella De Vil (CUST039) dengan total pengeluaran 3.000.000.000 
- James Bond (CUST046) dengan total pengeluaran 3.000.000.000 
- Indiana Jones (CUST009) dengan total pengeluaran 3.000.000.000 
- Bob The Builder (CUST002) dengan total pengeluaran 3.500.000.000 
- Diana Prince (CUST004) dengan total pengeluaran 2.500.000.000 
- Leia Organa (CUST012) dengan total pengeluaran 2.800.000.000 
- Tarzan Jungle (CUST020) dengan total pengeluaran 2.500.000.000 
- Buzz Lightyear (CUST026) dengan total pengeluaran 2.100.000.000 
- Katniss Everdeen Sr (CUST047) dengan total pengeluaran 2.000.000.000 
- Darth Vader (CUST040) dengan total pengeluaran 2.500.000.000 
- Lara Croft (CUST048) dengan total pengeluaran 1

In [7]:
print("user question:", user_question)

answer = crew_output.raw    # list of dict
print(json.dumps(answer, default=lambda x: float(x) if isinstance(x, Decimal) else x, indent=2))

user question: Siapa saja pelanggan yang total pengeluaran bulan lalu di atas 3 juta rupiah
"Berikut adalah daftar pelanggan yang total pengeluaran bulan lalu di atas 3 juta rupiah: \n- Iron Man (CUST069) dengan total pengeluaran 12.000.000\n- Tony Stark (CUST056) dengan total pengeluaran 15.000.000.000 \n- Queen Amidala (CUST017) dengan total pengeluaran 4.000.000.000 \n- Cruella De Vil (CUST039) dengan total pengeluaran 3.000.000.000 \n- James Bond (CUST046) dengan total pengeluaran 3.000.000.000 \n- Indiana Jones (CUST009) dengan total pengeluaran 3.000.000.000 \n- Bob The Builder (CUST002) dengan total pengeluaran 3.500.000.000 \n- Diana Prince (CUST004) dengan total pengeluaran 2.500.000.000 \n- Leia Organa (CUST012) dengan total pengeluaran 2.800.000.000 \n- Tarzan Jungle (CUST020) dengan total pengeluaran 2.500.000.000 \n- Buzz Lightyear (CUST026) dengan total pengeluaran 2.100.000.000 \n- Katniss Everdeen Sr (CUST047) dengan total pengeluaran 2.000.000.000 \n- Darth Vader (CUST